# Agent Notebook
This notebook demonstrates an agent workflow using autogen and models hosted on Ollama.

In [12]:
!pip install -U     autogen     autogen-ext[ollama]     jupytext markdownify     accelerate bitsandbytes


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports and constants

In [13]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat
# local fallback client if library missing
try:
    from autogen import OpenAIChatCompletionClient
except ImportError:
    class OpenAIChatCompletionClient:
        def __init__(self, model, base_url=None, api_key=None):
            self.model = model
            self.base_url = base_url
            self.api_key = api_key
        def chat(self, messages, temperature=0):
            raise NotImplementedError('OpenAIChatCompletionClient is unavailable')
import pathlib, datetime, uuid, subprocess, shlex, markdownify

REPO_ROOT = pathlib.Path.cwd()
LOG_DIR = REPO_ROOT / 'agent_logs'
LOG_DIR.mkdir(exist_ok=True, parents=True)
BASE_URL = 'http://docker-ai:11434/v1'


## Model clients

In [14]:
qwen14_client = OpenAIChatCompletionClient(
    model='qwen2.5-14b-1m',
    base_url=BASE_URL,
    api_key='ollama')

qwen32_client = OpenAIChatCompletionClient(
    model='qwen3-32b-5k_s',
    base_url=BASE_URL,
    api_key='ollama')

devstral_client = OpenAIChatCompletionClient(
    model='devstral-24b-5k_s',
    base_url=BASE_URL,
    api_key='ollama')

## Markdown logger

In [15]:
def log_markdown(task_id, role, content):
    ts = datetime.datetime.utcnow().isoformat()
    fn = LOG_DIR / f"{task_id}.md"
    if not fn.exists():
        with open(fn, 'w') as f:
            f.write(f'---\nid: {task_id}\ncreated: {ts}\n---\n\n')
    with open(fn, 'a') as f:
        f.write(f'### {ts} — {role}\n\n{markdownify.markdownify(content)}\n\n')


## Shell helper

In [16]:
def run_shell(cmd: str) -> str:
    out = subprocess.check_output(shlex.split(cmd), text=True, timeout=900, stderr=subprocess.STDOUT)
    return f'```shell\n$ {cmd}\n{out}\n```'


## Agent declarations

In [20]:
planner = AssistantAgent(
    name='planner',
    llm_config={'client': qwen14_client, 'temperature': 0.3},
    system_message=("You are a project planner. Break the user's request into a YAML list of atomic tasks. Stop when each sub-task can be executed in one short Python call or shell command inside the current Jupyter kernel."),
)

worker = AssistantAgent(
    name='worker',
    llm_config={'client': qwen32_client, 'temperature': 0},
    system_message='Execute the given atomic task and return result.')
worker.register_function(run_shell)

coder = AssistantAgent(
    name='coder',
    llm_config={'client': devstral_client, 'temperature': 0},
    system_message='You are a senior software engineer. Write, refactor, and debug code snippets as requested.')

reviewer = AssistantAgent(
    name='reviewer',
    llm_config={'client': qwen14_client, 'temperature': 0},
    system_message=("Evaluate the worker or coder output against the task description. If incorrect, respond with REVISE and instructions; otherwise APPROVED."),
)

agents = [planner, worker, coder, reviewer]
group = GroupChat(agents=agents, max_round=30)
proxy = UserProxyAgent(groupchat=group, human_input_mode='NEVER')


ValidationError: 2 validation errors for _LLMConfig
config_list.0.openai.model
  Field required [type=missing, input_value={'client': <__main__.Open...>, 'api_type': 'openai'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
config_list.0.openai.client
  Extra inputs are not permitted [type=extra_forbidden, input_value=<__main__.OpenAIChatCompl...t at 0x00000225D9CA7230>, input_type=OpenAIChatCompletionClient]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

## Driver function

In [ ]:
def run_agent(prompt: str):
    task_id = uuid.uuid4().hex[:8]
    log_markdown(task_id, 'USER', prompt)
    proxy.initiate_chat(prompt=prompt)
    for m in group.chat_history:
        log_markdown(task_id, m['role'], m['content'])
    return LOG_DIR / f'{task_id}.md'

## Example call

In [ ]:
run_agent('Generate Python code to scrape example.com daily and store results in SQLite …')

## Git auto-commit

In [ ]:
!git add agent_logs/*.md && (git diff --cached --quiet || git commit -m 'agent run') && git push

## Version info

In [ ]:
!pip freeze | grep -E '(autogen|transformers|ollama)'